In [419]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

from langdetect import detect
import translators.server as ts

import warnings
import tensorflow as tf
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

warnings.filterwarnings("ignore")

In [168]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36', 'Accept-Language':'pt-BR,pt;q=0.9,en;q=0.8'}
s = requests.Session()

In [169]:
def get_data(url, classe_name, tag_name):
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html, 'html.parser')
    texto = ''
    try:
        texto = soup.find_all(tag_name, class_= classe_name).text.split('\n')
        return texto
    
    except AttributeError:
        return []


In [420]:
input = 'GTX 2060'
text = input + ' amazon'
url = "https://www.google.com/search?q=" + text

request_result=s.get(url).content

soup = BeautifulSoup(request_result,"html.parser")

In [421]:
cont = 0

new_url = ''
reviews = []
produtos = []
for a in soup.find_all('a', href=True):    
    if 'www.amazon.com' in a['href'] and '/dp/' in a['href']:
        href888 = a['href']
        produtos.append(a.h3.text)
        new_url = href888.replace('/url?q=', '') # remove /url?q=
        url_coments = new_url.split('&')[0].replace('/dp/', '/product-reviews/').replace('.br', '')
        if '//'  in url_coments:
            url_coments = url_coments.split('//')[1]
        reviews.append('https://' + url_coments)

AttributeError: 'NoneType' object has no attribute 'text'

In [426]:
from math import ceil
coments = []
cont=0
for r in reviews:
    
    try:
        html = s.get(r + f'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1', headers=headers).content
        soup = BeautifulSoup(html, 'html.parser')

        a_soupi = soup.find_all('div', class_='a-row a-spacing-base a-size-base')
        
        if len(a_soupi) == 0:
            continue
        print(f"Analisando produto {produtos[cont]}")
        print(a_soupi[0].text.replace('\n', '').replace('  ', ''))
        i = int(a_soupi[0].text.replace('\n', '').replace('  ', '').replace(',', '').split('ratings ')[1].split(' ')[0])/10
        i = ceil(i)


        # print(f"{i} Comentários\n")

        for j in range(1, i):
            html = s.get(r + f'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber={j}', headers=headers).content
            a_soup = soup.find_all('div', class_='a-row a-spacing-small review-data')
            
            for a in a_soup:
                for coment in a.text.split('\n'):
                    if coment != '':
                        coments.append(coment)
        
    except:
        print('error')
        continue

    cont+=1
    print()

Analisando produto Placa de Video RTX2060 OC 6GB 192 Bit, Gigabyte, 9VN2060O6 ...
1,112 total ratings, 201 with reviews
Analisando produto Placa de Vídeo GALAX - GeForce RTX 2060, 1-Click OC, 6GB GDDR6
36 total ratings, 16 with reviews
Analisando produto Galax PLACA DE VIDEO GEFORCE RTX 2060 PLUS (1 CLICK
370 total ratings, 61 with reviews
Analisando produto MSI Gaming GeForce RTX 2060 6GB GDRR6 192-bit ... - Amazon
770 total ratings, 169 with reviews


In [424]:
coments_translate = []
print(f"Traduzindo {len(coments)} comentários...)")
for c in coments:
    try:
        detec = detect(c)
        if detec != 'en':
            len(c)
            coments_translate.append(ts.google(c, to_language='en'))
        else:
            coments_translate.append(c)
    except:
        try:
            coments_translate.append(ts.google(c, to_language='en'))
        except:
            
            continue
        continue

In [425]:
print(f'{len(coments_translate)} de {len(coments)} comentário traduzidos válidos')

428 de 428 comentário traduzidos válidos


In [427]:
coments

['Great product, excellent value and quick delivery',
 'Scheda video potente e ben costruita, le ventole svolgono il loro lavoro senza essere però troppo rumorose, è secondo me adatta al gaming il FHD a 144hz in molti giochi tranquillamente, attenzione però al prezzo.Se hai trovato utile la mia recensione sarebbe bello se lasciassi un bel 👍',
 'Top',
 "La scheda è perfetta. Le ventole girano alla perfezione, anche grazie al software di casa Gigabyte, Aorus Engine, creato appositamente per gestire appieno i carichi di lavoro delle GPU, oltre che al funzionamento delle ventole di raffreddamento.C'è da dire che essendo essa una scheda OC di fabbrica, arriva ad un boost di 2100 Mhz nel core interno e 1800 Mhz per la memoria GDDR6, di conseguenza scalda parecchio e giunge facilmente a temperature elevat. Se non si ha un case ben spazioso e ben ventilato prima ci si adoperi per facilitare il suo raffreddamento, altrimenti va downclockata e sarebbe un peccato date le prestazioni di questa sch

## Testando modelo

In [428]:
from sklearn.linear_model import LogisticRegression
import nltk

In [429]:
# importando a lista da nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords_en = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pichau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [430]:
import spacy.cli

spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [431]:
import en_core_web_sm

spc_en = en_core_web_sm.load()

In [432]:
df_alexa = pd.read_csv('amazon_alexa.tsv', sep = '\t')

In [433]:
positive = df_alexa[df_alexa['feedback'] == 1]
negative = df_alexa[df_alexa['feedback'] == 0]


In [434]:
df_alexa.drop(['date', 'rating', 'variation'], 1, inplace=True)


In [435]:
df_alexa

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1
...,...,...
3145,"Perfect for kids, adults and everyone in betwe...",1
3146,"Listening to music, searching locations, check...",1
3147,"I do love these things, i have them running my...",1
3148,Only complaint I have is that the sound qualit...,1


In [436]:
def limpa_texto(texto):
  '''(str) -> str
  Essa funcao recebe uma string, deixa tudo em minusculo, filtra apenas letras,
  retira stopwords, lemmatiza e retorna a string resultante.
  '''
  texto = texto.lower()

  texto = re.sub(r"[\W\d_]+", " ", texto)

  texto = [pal for pal in texto.split() if pal not in stopwords_en]

  spc_texto = spc_en(" ".join(texto))
  tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in spc_texto]
  
  return " ".join(tokens)

In [437]:
# Aplica a funcao nas reviews do dataset
df_alexa['verified_reviews'] = df_alexa['verified_reviews'].apply(limpa_texto)

## Vectorizer

In [438]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [439]:
texto = df_alexa['verified_reviews']
# Importando o TfidfVectorizer

# Instanciando o TfidfVectorizer
tfidf_vect = TfidfVectorizer()

# Vetorizando
X_tfidf = tfidf_vect.fit_transform(texto)

## Modelo

In [440]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf.toarray(), df_alexa['feedback'], test_size = 0.2, random_state = 42)

In [441]:
X_train.shape


(2520, 3103)

In [442]:
X_test.shape

(630, 3103)

In [443]:
# reglog = LogisticRegression()

# # Aplicando o modelo
# reglog.fit(X_train, y_train)

# # Predicao
# y_pred_test = reglog.predict(X_test)

classifier = tf.keras.models.Sequential()
classifier.add(tf.keras.layers.Dense(units = 10, activation='relu', input_shape=(3103,)))
classifier.add(tf.keras.layers.Dense(units = 10, activation='relu'))
classifier.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

In [444]:
classifier.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])


In [445]:
epochs_hist = classifier.fit(X_train, y_train, epochs=10)


Epoch 1/10
79/79 [==============================] - 1s 1ms/step - loss: 0.5827 - accuracy: 0.9183
Epoch 2/10
79/79 [==============================] - 0s 1ms/step - loss: 0.3264 - accuracy: 0.9210
Epoch 3/10
79/79 [==============================] - 0s 1ms/step - loss: 0.2374 - accuracy: 0.9210
Epoch 4/10
79/79 [==============================] - 0s 1ms/step - loss: 0.1993 - accuracy: 0.9210
Epoch 5/10
79/79 [==============================] - 0s 1ms/step - loss: 0.1565 - accuracy: 0.9282
Epoch 6/10
79/79 [==============================] - 0s 1ms/step - loss: 0.1198 - accuracy: 0.9516
Epoch 7/10
79/79 [==============================] - 0s 1ms/step - loss: 0.0904 - accuracy: 0.9702
Epoch 8/10
79/79 [==============================] - 0s 1ms/step - loss: 0.0687 - accuracy: 0.9806
Epoch 9/10
79/79 [==============================] - 0s 1ms/step - loss: 0.0546 - accuracy: 0.9861
Epoch 10/10
79/79 [==============================] - 0s 1ms/step - loss: 0.0452 - accuracy: 0.9885


In [446]:
y_pred_test = (classifier.predict(X_test) > 0.5).astype(int)
# y_pred_test = (y_pred_test > 0.5).astype(int)

20/20 [==============================] - 0s 711us/step


In [447]:
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       0.76      0.43      0.55        58
           1       0.94      0.99      0.96       572

    accuracy                           0.93       630
   macro avg       0.85      0.71      0.76       630
weighted avg       0.93      0.93      0.93       630



In [449]:
coments_translate

['Great product, excellent value and quick delivery',
 'Powerful and well -built video card, the fans carry out their work without being too noisy, it is in my opinion suitable for gaming the FHD at 144Hz in many games quietly, but be careful at the price. If you found my review useful would be nice if I lefta nice 👍 👍',
 'Top',
 'The card is perfect.The fans run perfectly, also thanks to the Gigabyte software, Aorus Engine, created specifically to fully manage the workloads of the GPUs, as well as the functioning of the cooling fans.Factory, reaches a 2100 MHz boost in the internal core and 1800 MHz for the GDDR6 memory, consequently he warms a lot and easily reaches elevated temperatures.If you do not have a well spacious and well -ventilated case, you first use it to facilitate its cooling, otherwise it goes downClockata and it would be a shame give the performance of this tab.',
 "I upgraded to this card from a GTX 970 which was struggling to run games at 1440p resolution.So far I 

In [450]:
c = tfidf_vect.transform(coments_translate).toarray()
predict = (classifier.predict(c) > 0.5).astype(int)

14/14 [==============================] - 0s 815us/step


In [451]:
cont_pos = 0
cont_neg = 0
for p in predict:
    if p == 1:
        cont_pos += 1
    else:
        cont_neg += 1

In [452]:
print(f'Positivos: {cont_pos} | Negativos: {cont_neg}')

Positivos: 422 | Negativos: 6
